# Building Q&A application using Knowledge Bases for Amazon Bedrock - Retrieve API and Langchain

### Context

With a knowledge base, you can securely connect foundation models (FMs) in Amazon Bedrock to your company
data for Retrieval Augmented Generation (RAG). Access to additional data helps the model generate more relevant,
context-speciﬁc, and accurate responses without continuously retraining the FM. All information retrieved from
knowledge bases comes with source attribution to improve transparency and minimize hallucinations. For more information on creating a knowledge base using console, please refer to this [post](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html).

In this notebook, we will dive deep into building Q&A application using Retrieve API provided by [Knowledge Bases for Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html) and [LangChain](https://python.langchain.com/docs/get_started/introduction). We will query the knowledge base to get the desired number of document chunks based on similarity search, integrate it with LangChain retriever and use Anthropic Claude instant model for answering questions.


### Pattern

We can implement the solution using Retreival Augmented Generation (RAG) pattern. RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. Here, we are performing RAG effectively on the knowledge base created in the previous notebook or using console. 

### Pre-requisite

Before being able to answer the questions, the documents must be processed and stored in knowledge base.

1. Load the documents into the knowledge base by connecting your s3 bucket (data source). 
2. Ingestion - Knowledge base will split them into smaller chunks (based on the strategy selected), generate embeddings and store it in the associated vectore store and notebook [0_create_ingest_documents_test_kb.ipynb](./0\_create_ingest_documents_test_kb.ipynb) takes care of it for you.

![data_ingestion.png](../../imgs/52-rag-with-external-data.png)


#### Notebook Walkthrough

For our notebook we will use the `Retreive API` provided by Knowledge Bases for Amazon Bedrock which converts user queries into
embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom
workﬂows on top of the semantic search results. The output of the `Retrieve API` includes the `retrieved text chunks`, the `location type` and `URI` of the source data, as well as the relevance `scores` of the retrievals. 


We will then use the `RetrievalQAChain` provided by LangChain, add `RetreiverAPI` as a `retriever` in the chain. This chain will then automatically augment the text chunks being generated with the original prompt and pass it through the `anthropic.claude-instant-v1` model.


### Ask question
We will use the following workflow for this notebook. 

![retrieve.png](../../imgs/chatbot_lang.png)


### USE CASE:

#### Dataset

In this example, you will use several years of Amazon's Letter to Shareholders as a text corpus to perform Q&A on. This data is already ingested into the Kknowledge Bases for Amazon Bedrock. You will need the `knowledge base id` to run this example.

### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠

### Setup

To run this notebook you would need to install dependencies, and LangChain and update boto3, botocore for accessing the newly released Query API provided by Knowledge Bases for Amazon Bedrock.


In [1]:
%pip install --upgrade pip
%pip install boto3==1.33.2 --force-reinstall --quiet
%pip install botocore==1.33.2 --force-reinstall --quiet
%pip install langchain>=0.1.11
%pip install pypdf==4.1.0
%pip install langchain-community faiss-cpu==1.8.0 tiktoken==0.6.0 sqlalchemy==2.0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.33.1 requires botocore==1.34.119, but you have botocore 1.33.13 which is incompatible.
awscli 1.33.1 requires s3transfer<0.11.0,>=0.10.0, but you have s3transfer 0.8.2 which is incompatible.
opensearch-py 2.3.1 requires urllib3<2,>=1.21.1, but you have urllib3 2.0.7 which is incompatible.
sagemaker 2.219.0 requires boto3<2.0,>=1.33.3, but you have boto3 1.33.2 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.1 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,

#### Restart the kernel with the updated packages that are installed through the dependencies above

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [5]:
store -r kb_id

### Follow the steps below to set up necessary packages

1. Import the necessary libraries for creating `bedrock-runtime` for invoking foundation models and `bedrock-agent-runtime` client for using Retrieve API provided by Knowledge Bases for Amazon Bedrock. 
2. Import Langchain for: 
   1. Initializing bedrock model  `anthropic.claude-v2` as our large language model to perform query completions using the RAG pattern. 
   2. Initialize Langchain retriever integrated with knowledge bases. 
   3. Later in the notebook we will wrap the LLM and retriever with `RetrieverQAChain` for building our Q&A application.

In [6]:
import boto3
import pprint
from botocore.client import Config
from langchain.llms.bedrock import Bedrock
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever

pp = pprint.PrettyPrinter(indent=2)
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config
                              )

model_kwargs_claude = {
    "temperature": 0,
    "top_k": 10,
    "max_tokens_to_sample": 3000
}

llm = Bedrock(model_id="anthropic.claude-instant-v1",
              model_kwargs=model_kwargs_claude,
              client = bedrock_client,)

### Retrieve API: Process flow 

Create a `AmazonKnowledgeBasesRetriever` object from LangChain which will call the `Retreive API` provided by Knowledge Bases for Amazon Bedrock which converts user queries into
embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom
workﬂows on top of the semantic search results. The output of the `Retrieve API` includes the the `retrieved text chunks`, the `location type` and `URI` of the source data, as well as the relevance `scores` of the retrievals. 

In [7]:

retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 4}},
        # endpoint_url=endpoint_url,
        # region_name="us-east-1",
        # credentials_profile_name="<profile_name>",
    )
docs = retriever.get_relevant_documents(
        query="By what percentage did AWS revenue grow year-over-year in 2022?"
    )
pp.pprint(docs)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[ Document(page_content='They estimate that, in 2020, third-party seller profits from selling on Amazon were between $25 billion and $39 billion, and to be conservative here I’ll go with $25 billion.   For customers, we have to break it down into consumer customers and AWS customers.   We’ll do consumers first. We offer low prices, vast selection, and fast delivery, but imagine we ignore all of that for the purpose of this estimate and value only one thing: we save customers time.   Customers complete 28% of purchases on Amazon in three minutes or less, and half of all purchases are finished in less than 15 minutes. Compare that to the typical shopping trip to a physical store – driving, parking, searching store aisles, waiting in the checkout line, finding your car, and driving home. Research suggests the typical physical store trip takes about an hour. If you assume that a typical Amazon purchase takes 15 minutes and that it saves you a couple of trips to a physical store a week, tha

`score`: You can view the associated score of each of the text chunk that was returned which depicts its correlation to the query in terms of how closely it matches it.

### Prompt specific to the model to personalize responses 

Here, we will use the specific prompt below for the model to act as a financial advisor AI system that will provide answers to questions by using fact based and statistical information when possible. We will provide the `Retrieve API` responses from above as a part of the `{context}` in the prompt for the model to refer to, along with the user `query`.  

In [8]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
    Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
    Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    <context>
    {context}
    </context>

    <question>
    {question}
    </question>

    The response should be specific and use statistics or numbers when possible.

    Assistant:"""
claude_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                                input_variables=["context","question"])

In [9]:
# fetch context from the response
def get_contexts(docs):
    contexts = []
    for retrievedResult in docs: 
        contexts.append(retrievedResult.page_content)
    return contexts

In [10]:
contexts = get_contexts(docs)
pp.pprint(contexts)

[ 'They estimate that, in 2020, third-party seller profits from selling on '
  'Amazon were between $25 billion and $39 billion, and to be conservative '
  'here I’ll go with $25 billion.   For customers, we have to break it down '
  'into consumer customers and AWS customers.   We’ll do consumers first. We '
  'offer low prices, vast selection, and fast delivery, but imagine we ignore '
  'all of that for the purpose of this estimate and value only one thing: we '
  'save customers time.   Customers complete 28% of purchases on Amazon in '
  'three minutes or less, and half of all purchases are finished in less than '
  '15 minutes. Compare that to the typical shopping trip to a physical store – '
  'driving, parking, searching store aisles, waiting in the checkout line, '
  'finding your car, and driving home. Research suggests the typical physical '
  'store trip takes about an hour. If you assume that a typical Amazon '
  'purchase takes 15 minutes and that it saves you a couple of

### Initiate the user prompt and response via the LLM

Here, we are going to format our prompt using the context generated by the retrieve API as well as the user query to get the final response that we will use to evaluate generated answers using LLaMaIndex

In [11]:
query = "By what percentage did AWS revenue grow year-over-year in 2022?"
prompt = claude_prompt.format(context=contexts, 
                                 question=query)

In [12]:
response = llm(prompt)
pp.pprint(response)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


(' Based on the context provided:\n'
 '\n'
 'AWS revenue grew 29% year-over-year in 2022 on a $62 billion revenue base in '
 '2021. However, the context mentions that AWS is now facing some short-term '
 'headwinds and softening growth due to macroeconomic conditions.')


## Integrating the retriever and the LLM defined above with `RetrievalQA` Chain to build the Q&A application.

In [13]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": claude_prompt}
)

In [14]:
answer = qa(query)
pp.pprint(answer)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{ 'query': 'By what percentage did AWS revenue grow year-over-year in 2022?',
  'result': ' Based on the information provided:\n'
            '\n'
            'AWS revenue grew 29% year-over-year in 2022 on a $62 billion '
            'revenue base in 2021. So AWS revenue growth in 2022 was 29%.',
  'source_documents': [ Document(page_content='They estimate that, in 2020, third-party seller profits from selling on Amazon were between $25 billion and $39 billion, and to be conservative here I’ll go with $25 billion.   For customers, we have to break it down into consumer customers and AWS customers.   We’ll do consumers first. We offer low prices, vast selection, and fast delivery, but imagine we ignore all of that for the purpose of this estimate and value only one thing: we save customers time.   Customers complete 28% of purchases on Amazon in three minutes or less, and half of all purchases are finished in less than 15 minutes. Compare that to the typical shopping trip to a physical

## Next Steps
If you are interested in evaluating your RAG application, for sample code, try notebook: 
"[customized-rag-retrieve-api-titan-lite-evaluation](https://github.com/aws-samples/amazon-bedrock-samples/blob/bedrock-kb-images-update/knowledge-bases/4_customized-rag-retrieve-api-titan-lite-evaluation.ipynb/) where we are using `Amazon Titan Lite` model for generating responses and `Anthropic Claude V2` for evaluating response. 